### Gathering Data from NamUs

NamUs is the National Missing and Unidentified Persons System, which is financed by the United States Department of Justice. NamUs does not provide an API, however it does provide a searchable interface.

In this notebook, we will use Selenium with beautifulSoup to retrieve and save data from NamUs as CSV.

In [1]:
# Import libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import getopt, re, sys, time, os

In [2]:
# initialize global driver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

In [3]:
# Get path to the repository's data folder
path = "/".join(os.getcwd().split("/")[0:-1]) + "../web-scraping/namus/unclaimed_states.csv"
print(path)


../web-scraping/namus/unclaimed_states.csv


In [4]:
# Constants
CASE_NUMBER_KEY = 'Case Number'

INFO_COLUMNS = [
    'Case Number',
    'DBF',
    'Last Name',
    'First Name',
    'Sex',
    'Race/Ethnicity',
    'City',
    'County',
    'State', 
    'Date Modified'
]
MAX_ROWS_PER_PAGE = 100

In [5]:
def add_filters(filters):
    if 'states' in filters: location_filter(filters['states'])
    if 'date_elem' in filters: filtering_by_date(filters)

In [6]:
###

def filtering_by_date(date):
    print('Adding date filters...')

    section_on_circumstances = driver.find_element_by_id('Circumstances')
    operand_box = section_on_circumstances.find_elements_by_tag_name('date-range-input')[1].find_elements_by_tag_name('select')[0]
    Select(operand_box).select_by_visible_text(date['date_elem'])

    time.sleep(.5)

    month_box = section_on_circumstances.find_elements_by_tag_name('date-range-input')[1].find_elements_by_tag_name('select')[1]
    Select(month_box).select_by_visible_text(date['month'])

    day_box = section_on_circumstances.find_elements_by_tag_name('date-range-input')[1].find_elements_by_tag_name('select')[2]
    Select(day_box).select_by_visible_text(date['day'])

    year_box = section_on_circumstances.find_elements_by_tag_name('date-range-input')[1].find_elements_by_tag_name('select')[3]
    Select(year_box).select_by_visible_text(date['year'])

In [7]:
# find state filter
def location_filter(states):
    print('Fetching selected states to filter...')
    section_on_circumstances = driver.find_element_by_id('Circumstances')
    labels_in_section = section_on_circumstances.find_elements_by_tag_name('label')

    state_input_box = None

    for label in labels_in_section:
        if (label.text == "State"):
            state_input_box = label.find_element_by_tag_name('input')
            # add state filter
            for state in states:
                state_input_box.send_keys(state)
                state_input_box.send_keys(Keys.ENTER)
                


In [8]:
def records():
    print('Collecting data...')

    # navigate to list view
    driver.find_element_by_xpath("//i[@class=\"icon-list\"]").click()
    time.sleep(1.5)

    df_info = pd.DataFrame(columns=INFO_COLUMNS)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    rows = soup.find('div', class_='ui-grid-canvas').contents

    for row in rows:
        if row != ' ':
            cells = row.find_all('div', class_='ui-grid-cell-contents')
            cells_text = map(lambda cell: cell.text.strip(), cells)
            df_new_info = pd.DataFrame([list(cells_text)], columns=INFO_COLUMNS)
            df_info = pd.concat([df_info, df_new_info], ignore_index=True)
    
    return df_info

In [9]:
def page_counts():
    print('Counting the amount of pages...')
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    page_num_data = soup.find('nav', {'aria-label': 'Page Selection'}).find('span').text
    index_of_slash = re.search('/', page_num_data).span()[1]
    page_numbers = int(page_num_data[index_of_slash:].strip())

    return page_numbers

In [10]:
def next_page():
    print('navigating to the next page...')
    time.sleep(5)

    try:
        driver.find_element_by_xpath("//i[@class=\"icon-triangle-right\"]").click()
    except:
        print('last page completed...')

In [11]:
def parse_args(argv):
    help_message = """
    Use: --states = New York
        Allows a comma-separated list, such as (--states=Oregon, California).
        Date of Last Interaction:-date= can search for dates greater or smaller than a certain date
                    Example:    --date=">=May-5-1995" 
                                --date="<=February-12-1997" 
        -h :        Displays a help screen; alternatively, use --help
    """

    filters = {}

    try:
        opts, args = getopt.getopt(argv,'h',['help', 'states=', 'date='])
    except getopt.GetoptError:
        print(help_message)
        sys.exit(2)

    for opt, arg in opts:
        if opt in ('-h','--help'):
            print(help_message)
            sys.exit()
        if opt == '--states':
            filters['states'] = arg.split(',')
        if opt == '--date':
            filters['date_elem'] = arg[:2]
            filters['month'] = arg[2:].split('-')[0]
            filters['day'] = arg[2:].split('-')[1]
            filters['year'] = arg[2:].split('-')[2]

    return filters

In [12]:
# show 100 results at a time
def rows_to_show(num_rows):
    print(f'Setting {MAX_ROWS_PER_PAGE} rows per page...')
    dropdown_selection_results = driver.find_element_by_xpath("//label/span[contains(text(),'Results')]/following-sibling::select")
    Select(dropdown_selection_results).select_by_value(f'{num_rows}')
    time.sleep(1.5)

In [13]:
def search():
    print('Searching...')
    page_results = driver.find_element_by_class_name('search-criteria-container')
    search_actions = page_results.find_element_by_class_name('search-criteria-container-actions').find_elements_by_tag_name('input')
    search_actions[1].click()
    time.sleep(1.5)

In [14]:
def main(argv):
    filters = parse_args(
        argv=['--states=Alabama, Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, District of Columbia, Florida, Georgia, Guam, Hawaii, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland, Massachusetts, Michigan, Minnesota, Mississippi, Missouri, Montana, Nebraska, Nevada, New Hampshire, New Jersey, New Mexico, North Carolina, North Dakota, Northern Mariana Islands, Ohio, Oklahoma, Oregon, Pennsylvania, Puerto Rico, Rhode Island, South Carolina, South Dakota, Tennessee, Texas, Virgin Islands, Utah, Vermont, Virginia, Washington, West Virginia, Wisconsin, Wyoming']
        )
    
    print('Navigating to namus.gov...')
    driver.get("https://www.namus.gov/UnclaimedPersons/Search")

    add_filters(filters)
    search()
    print("Starting case processing")

    rows_to_show(MAX_ROWS_PER_PAGE)
    page_numbers = page_counts()
    df_info = pd.DataFrame(columns=INFO_COLUMNS)

    try:
        for page in range(page_numbers):
            print(f'Gathering page {page}...')
            new_df = records()
            df_info = pd.concat([df_info, new_df], ignore_index=True)
            next_page()
    except Exception as e:
        print(f'Exception thrown. Creating a csv file from existing data: {path}')
        df_info.to_csv(path, index=False, encoding='utf-8')
        #driver.quit()
        print(e)
    
    # Output collected data to the "web-scraping" folder

    print(f'Saving gathered data to csv: {path}')
    df_info.to_csv(path, index=False, encoding='utf-8')
    #driver.quit()

    print('Scraping completed')
    

if __name__ == '__main__':
    main(sys.argv[1:])

Navigating to namus.gov...
Fetching selected states to filter...
Searching...
Starting case processing
Setting 100 rows per page...
Counting the amount of pages...
Gathering page 0...
navigating to the next page...
Gathering page 1...
navigating to the next page...
Gathering page 2...
navigating to the next page...
Gathering page 3...
navigating to the next page...
Gathering page 4...
navigating to the next page...
Gathering page 5...
navigating to the next page...
Gathering page 6...
navigating to the next page...
Gathering page 7...
navigating to the next page...
Gathering page 8...
navigating to the next page...
Gathering page 9...
navigating to the next page...
Gathering page 10...
navigating to the next page...
Gathering page 11...
navigating to the next page...
Gathering page 12...
navigating to the next page...
Gathering page 13...
navigating to the next page...
Gathering page 14...
navigating to the next page...
Gathering page 15...
navigating to the next page...
Gathering page

In [15]:
driver.quit()

In [16]:
df_unclaimed = pd.read_csv('../web-scraping/namus/unclaimed_states.csv')



In [17]:
df_unclaimed

Case Number         DBF  Last Name   First Name     Sex  \
0       UCP93119  07/05/2022      Jason         Jean    Male   
1       UCP93208  06/29/2022    Lehmann       Ursula  Female   
2       UCP93351  06/25/2022     Fowler       Donald    Male   
3       UCP93244  06/19/2022     Knutie  Christopher    Male   
4       UCP93166  06/16/2022     Fisher        Laron    Male   
...          ...         ...        ...          ...     ...   
7498    UCP66090          --  Scripture    Elizabeth      --   
7499    UCP66093          --   Heaertel       Jeremy      --   
7500    UCP66117          --      Jones      Lillian      --   
7501    UCP66116          --      Welch      Lillian      --   
7502    UCP66177          --       Horn     Kathleen      --   

                Race/Ethnicity        City     County       State  \
0     Black / African American        Reno     Washoe      Nevada   
1            White / Caucasian        Reno     Washoe      Nevada   
2            White / Caucasian  Birmingham  Jefferson     Alabama   
3     Black / African American   Knoxville       Knox   Tennessee   
4     Black / African American     Houston     Harris       Texas   
...                        ...         ...        ...         ...   
7498                        --          --     Yakima  Washington   
7499                        --          --     Yakima  Washington   
7500                        --          --     Yakima  Washington   
7501                        --          --     Yakima  Washington   
7502                        --          --     Yakima  Washington   

     Date Modified  
0       07/07/2022  
1       07/11/2022  
2       07/14/2022  
3       07/12/2022  
4       07/08/2022  
...            ...  
7498    04/04/2020  
7499    04/04/2020  
7500    04/05/2020  
7501    04/05/2020  
7502    04/06/2020  

[7503 rows x 10 columns]